<a href="https://colab.research.google.com/github/mahb97/Wakeifier/blob/main/Wakefyer_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wakefyer (Colab) — LoRA fine-tune with conservative text handling


Runnit: from top to bottom except of all the cells that CLEARLY DONT WORK.
Over-clean the Wake and you'll make Joyce turn in his grave (he told me this via the tarot cards): by default, doesn't normalise or strip diacritics (ultra-light cleaning toggle is included to fill space, dont use, why would you do that)


In [1]:
!pip -q install "transformers==4.44.2" "peft==0.12.0" "accelerate==0.34.2" "datasets==2.20.0" "sentencepiece==0.2.0"
!pip install -U "transformers>=4.46.0"
import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True,max_split_size_mb:64"
torch.backends.cuda.matmul.allow_tf32 = True

# this will moan about compatibility...



  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [2]:
import transformers; print(transformers.__version__)


4.57.1


**A working cell for broke girls with no mula for colab and who dont wanna chug money into the firey pit that is hell that is google that is oppression.**

The Wakefyer fine-tuning architecture constitutes a hybridized LoRA-based low-rank adaptation pipeline for stylistic transformation within a bfloat16/fp16 mixed-precision environment. Its design paradigm is deliberately antithetical to large-scale quantized optimization frameworks such as bitsandbytes, privileging instead interpretability, traceability, and computational minimalism. Conceptually, it operates as a micro-epistemic engine: a system whose purpose is not semantic fidelity but stylistic distortion, functioning as a textual “entropy amplifier” with respect to Finnegans Wake’s logorrheic poetics.

Text ingestion follows a non-normalizing, non-canonical pipeline, thereby maintaining orthographic heterogeneity as an essential stylistic feature. The raw Joycean corpus is tokenized through a fast SentencePiece model and segmented using overlapping sliding windows defined by max_len and stride. This overlap emulates a continuum of narrativity (each window partially repeating the previous one) producing a computational analogue of Joyce’s recursive syntactic rhythm. No linguistic cleaning or lemmatization occurs; lexical noise is treated as signal. The preprocessing stage therefore performs not sanitization but syntactic fractalization, generating a dataset of semi-redundant micro-contexts to condition the model’s representational drift.

In [3]:
from google.colab import drive
drive.mount('/content/drive')
OUT_DIR = "/content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb"

!ls -lh /content | grep FinnegansWake
!head -n 5 /content/FinnegansWake.txt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
-rw-r--r-- 1 root root 1.4M Oct 16 20:11 FinnegansWake.txt
﻿* A Distributed Proofreaders Canada eBook *

This eBook is made available at no cost and with very few
restrictions. These restrictions apply only if (1) you make
a change in the eBook (other than alteration for different


In [4]:
# -*- coding: utf-8 -*-
"""
because code should also stream-of-consciousness. avoid the binary bore.

Colab prep (fresh runtime):
!pip -q install "transformers==4.44.2" "peft==0.12.0" "accelerate==0.34.2" "datasets==2.20.0" "sentencepiece==0.2.0"
"""
import os, random, sys, gc
from pathlib import Path
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
)
from peft import LoraConfig, get_peft_model

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TRAIN_TEXT = "/content/FinnegansWake.txt"           # gospel
OUT_DIR    = "/content/wakefyer_out_nobnb"          # the shards of my heart
max_len    = 256       # isuesssss
stride     = 128       # overlapsss
epochs     = 1         # binary bore
lr         = 1e-4      # gentle
batch_size = 1         # because T4s have fragile little snowflake egos
grad_accum = 32        # pretends the batch is bigger; shh....(keeps a T4 alive when fine-tuning a 1B-parameter model. We can cry about this collectively, meet me under the moon.)
log_steps  = 10
eval_ratio = 0.02
seed       = 42
import numpy as np
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# name my daughter LoRA
lora_r = 16
lora_alpha = 32
lora_dropout = 0.05
lora_targets = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

# Incantations against OOM
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:64")
torch.backends.cuda.matmul.allow_tf32 = True
torch.cuda.empty_cache()

def panic(msg):
    print(f"\n[Wakefyer shrieks] {msg}\n", file=sys.stderr)

# Sanity bells and whistles (is there a GPU? is it sulking?)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# Validate the gospel
if not Path(TRAIN_TEXT).exists():
    panic(f"the gospel'{TRAIN_TEXT}' is missing.")
    raise SystemExit(2)

raw_txt = Path(TRAIN_TEXT).read_text(encoding="utf-8", errors="ignore")
if len(raw_txt.strip()) < 1000:
    panic("Joyce did not write haikus.")

bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

# summons the devil (no batsandbites; just vibessss coz im a witch)

if len(raw_txt.strip()) < 1000:
    panic("Joyce did not write haikus.")

import re
punct = r"[!?.,:;·—–\-…]*"

thunderwords = [
    "bababadalgharaghtakamminarronnkonnbronntonnerronntuonnthunntrovarrhounawnskawntoohoohoordenenthurnuk",
    "Perkodhuskurunbarggruauyagokgorlayorgromgremmitghundhurthrumathunaradidillifaititillibumullunukkunun",
    "klikkaklakkaklaskaklopatzklatschabattacreppycrottygraddaghsemmihsammihnouithappluddyappladdypkonpkot",
    "Bladyughfoulmoecklenburgwhurawhorascortastrumpapornanennykocksapastippatappatupperstrippuckputtanach",
    "Thingcrooklyexineverypasturesixdixlikencehimaroundhersthemaggerbykinkinkankanwithdownmindlookingated",
    "Lukkedoerendunandurraskewdylooshoofermoyportertooryzooysphalnabortansporthaokansakroidverjkapakkapuk",
    "Bothallchoractorschumminaroundgansumuminarumdrumstrumtruminahumptadumpwaultopoofoolooderamaunsturnup",
    "Pappappapparrassannuaragheallachnatullaghmonganmacmacmacwhackfalltherdebblenonthedubblandaddydoodled",
    "husstenhasstencaffincoffintussemtossemdamandamnacosaghcusaghhobixhatouxpeswchbechoscashlcarcarcaract",
    "Ullhodturdenweirmudgaardgringnirurdrmolnirfenrirlukkilokkibaugimandodrrerinsurtkrinmgernrackinarockar",
]

def normalize_thunderwords(text: str, words: list[str]) -> str:
    text = re.sub(r"-\s*\n\s*", "", text)
    text = re.sub(r"\n+", " ", text)
    for w in words:
        pattern = re.compile(rf"(?<!\w){re.escape(w)}{punct}")
        text = pattern.sub(w, text)
    return re.sub(r"\s{2,}", " ", text).strip()

raw_txt = normalize_thunderwords(raw_txt, thunderwords)

# tell me more about your token magic

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

num_added = tok.add_tokens(thunderwords, special_tokens=False)
print("Added tokens:", num_added)
for w in thunderwords:
    toks = tok(w, add_special_tokens=False).input_ids
    print(len(toks), "→", w[:24] + ("…" if len(w)>24 else ""))  # expect binary bore

# Sanity B or menty b
from collections import Counter

tid_map = {w: tok.convert_tokens_to_ids(w) for w in thunderwords}

encoded = tok(raw_txt, add_special_tokens=False).input_ids
freq = Counter(encoded)

for w in thunderwords:
    t_id = tid_map[w]
    in_text = raw_txt.count(w)              # string-level occurrences
    as_single = freq.get(t_id, 0)           # times captured as the single new token
    print(f"{w[:24]}…  in_text={in_text}  as_single_token={as_single}")
    if in_text != as_single:
        print("  ↳ mismatch: normalise spelling/punct or update the list.")
    assert all(len(tok(w, add_special_tokens=False).input_ids) == 1 for w in thunderwords)

from transformers import AutoConfig
config = AutoConfig.from_pretrained(BASE_MODEL)
config.pad_token_id = tok.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    config=config,
    torch_dtype=torch.bfloat16 if bf16_ok else torch.float16,
    device_map="auto",
)
model.config.use_cache = False
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()

if num_added > 0:
    model.resize_token_embeddings(len(tok))

model.config._attn_implementation = "eager"

lora = LoraConfig(
    r=lora_r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
    target_modules=lora_targets, bias="none", task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora)

emb = model.get_input_embeddings()
emb.weight.requires_grad = True
if hasattr(model, "lm_head") and hasattr(model.lm_head, "weight"):
    model.lm_head.weight.requires_grad = True

model.print_trainable_parameters()

# pace urself
ids = tok(raw_txt).input_ids

def sliding_windows(txt_ids, block_size=max_len, step=stride):
    if block_size <= 0:
        panic("max_len is non‑positive. Even Joyce needs at least a token.")
        raise ValueError("max_len must be > 0")
    items = []
    # dont trip bro
    for start in range(0, max(1, len(txt_ids) - block_size), step):
        chunk = txt_ids[start:start+block_size]
        if not chunk:
            continue
        items.append({"input_ids": chunk, "attention_mask": [1]*len(chunk)})
    # what failure looks like when its not me
    if not items:
        chunk = txt_ids[:block_size]
        if not chunk:
            panic("After tokenization, we got zero tokens. Perhaps the tokenizer dont like Joyce.")
            raise ValueError("Empty token sequence")
        items = [{"input_ids": chunk, "attention_mask": [1]*len(chunk)}]
    return Dataset.from_list(items)


ids = tok(raw_txt).input_ids
ds = sliding_windows(ids, block_size=max_len, step=stride)
n = len(ds)
print("Windows:", n)
if n < 10:
    print("Consider lowering max_len or adding text for more windows.")


test_n = max(1, int(n * eval_ratio)) if n > 1 else 1
splits = ds.train_test_split(test_size=test_n, seed=seed) if n > 1 else {"train": ds, "test": ds}
train_ds, eval_ds = splits["train"], splits["test"]
print("Train windows:", len(train_ds), "Eval windows:", len(eval_ds))

# what do you know about masking
collator = DataCollatorForLanguageModeling(tok, mlm=False)

# Actual Life
args = TrainingArguments(
    output_dir=OUT_DIR,

    # I am a party
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,

    # Inside of my head
    learning_rate=lr,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,

    # Inside of my house
    logging_steps=log_steps,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=5,

    # It's just not that much fun having fun when you don't wanna have fun
    eval_steps=200,

    # precision
    bf16=bf16_ok,               # probably False on T4
    fp16=(not bf16_ok),

    # misc
    save_safetensors=True,
    report_to=[],
)

# Compatibility shim
import accelerate
from accelerate import Accelerator

try:
    if "keep_torch_compile" not in Accelerator.unwrap_model.__code__.co_varnames:
        _orig_unwrap = Accelerator.unwrap_model
        def _unwrap(self, model, *args, **kwargs):
            # Ignore extra kwargs like keep_torch_compile
            return _orig_unwrap(self, model)
        Accelerator.unwrap_model = _unwrap
        print("Patched accelerate.Accelerator.unwrap_model to ignore extra kwargs.")
except Exception as e:
    print("Could not patch accelerate.unwrap_model:", e)

from accelerate import Accelerator
Accelerator.unscale_gradients = lambda self, optimizer=None: None

import transformers.trainer_pt_utils as pt_utils
pt_utils.get_grad_scaler = lambda *a, **k: None

# light at the end of the tunnel
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collator,
)

from glob import glob

try:
    ckpts = sorted(glob(f"{OUT_DIR}/checkpoint-*"), key=lambda p: int(p.split("-")[-1]))
    last_ckpt = ckpts[-1] if ckpts else None
    trainer.train(resume_from_checkpoint=last_ckpt)
except RuntimeError as e:
    # likely OOM
    panic(f"Runtime groan: {e}")
    panic("Try: lower max_len (192 or 128), set model.config._attn_implementation='eager', increase grad_accum, or reduce LoRA r/alpha.")
    raise

adapter_dir = Path(OUT_DIR) / "adapter"
token_dir = Path(OUT_DIR) / "tokenizer"
adapter_dir.parent.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_dir, safe_serialization=True)
tok.save_pretrained(token_dir)
print("Saved:", adapter_dir, token_dir)

# a test for meee
prompt = "riverrun, past Eve and Adam's,"
inputs = tok(prompt, return_tensors="pt").to(model.device)
model.eval()
out = model.generate(
    **inputs,
    max_new_tokens=80,
    do_sample=True,
    temperature=1.05,
    top_p=0.92,
    repetition_penalty=1.05,
    pad_token_id=tok.eos_token_id,
)
print("\n=== SAMPLE ===\n", tok.decode(out[0], skip_special_tokens=True))

Torch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Added tokens: 10
1 → bababadalgharaghtakammin…
1 → Perkodhuskurunbarggruauy…
1 → klikkaklakkaklaskaklopat…
1 → Bladyughfoulmoecklenburg…
1 → Thingcrooklyexineverypas…
1 → Lukkedoerendunandurraske…
1 → Bothallchoractorschummin…
1 → Pappappapparrassannuarag…
1 → husstenhasstencaffincoff…
1 → Ullhodturdenweirmudgaard…


Token indices sequence length is longer than the specified maximum sequence length for this model (396323 > 2048). Running this sequence through the model will result in indexing errors
`torch_dtype` is deprecated! Use `dtype` instead!


bababadalgharaghtakammin…  in_text=1  as_single_token=0
  ↳ mismatch: normalise spelling/punct or update the list.
Perkodhuskurunbarggruauy…  in_text=1  as_single_token=0
  ↳ mismatch: normalise spelling/punct or update the list.
klikkaklakkaklaskaklopat…  in_text=1  as_single_token=0
  ↳ mismatch: normalise spelling/punct or update the list.
Bladyughfoulmoecklenburg…  in_text=1  as_single_token=1
Thingcrooklyexineverypas…  in_text=1  as_single_token=1
Lukkedoerendunandurraske…  in_text=1  as_single_token=1
Bothallchoractorschummin…  in_text=1  as_single_token=1
Pappappapparrassannuarag…  in_text=1  as_single_token=1
husstenhasstencaffincoff…  in_text=1  as_single_token=0
  ↳ mismatch: normalise spelling/punct or update the list.
Ullhodturdenweirmudgaard…  in_text=1  as_single_token=1


...

Windows: 3095
Train windows: 3034 Eval windows: 61
Patched accelerate.Accelerator.unwrap_model to ignore extra kwargs.


	save_steps: 50 (from args) != 200 (from trainer_state.json)


Step,Training Loss


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Saved: /content/wakefyer_out_nobnb/adapter /content/wakefyer_out_nobnb/tokenizer

=== SAMPLE ===
 riverrun, past Eve and Adam's, the trolls have disappeared!

The group continues, looking for any signs of the creatures. They finally stumble upon a small village made entirely out of rock, with only a few huts dotting the place. The huts have doors, but there are no people in sight.

John sighs and turns to Lily, "We may have outsmart


In [13]:
# HARD RESET VRAM
import gc, torch
for name in ["trainer","model","base"]:
    if name in globals():
        del globals()[name]
gc.collect(); torch.cuda.empty_cache()
try:
    torch.cuda.reset_peak_memory_stats()
except Exception:
    pass

# env knobs that help fragmentation
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:64"

# reload token magic
from pathlib import Path
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from peft import PeftModel

OUT_DIR    = "/content/wakefyer_out_nobnb"
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tok = AutoTokenizer.from_pretrained(Path(OUT_DIR,"tokenizer"), use_fast=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token
tok.padding_side = "right"

# low CPU mem
config = AutoConfig.from_pretrained(BASE_MODEL)
config.pad_token_id = tok.eos_token_id

offload_dir = "/content/offload"
os.makedirs(offload_dir, exist_ok=True)

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    config=config,
    torch_dtype=torch.float16,          # use 'torch_dtype' or 'dtype' depending on version
    low_cpu_mem_usage=True,             # stream shards; lower spikes
    device_map="auto",                  # let HF place layers
    offload_folder=offload_dir,         # spill excess to disk if needed
    # max_memory can further guard placement:
    # max_memory={0: "14GiB", "cpu": "30GiB"},
)

# resize to tokenizer
old_vocab = base.get_input_embeddings().num_embeddings
new_vocab = len(tok)
if old_vocab != new_vocab:
    try:
        base.resize_token_embeddings(new_vocab, mean_resizing=True)
    except TypeError:
        base.resize_token_embeddings(new_vocab)
    base.config.vocab_size = new_vocab
print(f"Resized vocab: {old_vocab} -> {new_vocab}")

# calling Lora
model = PeftModel.from_pretrained(base, Path(OUT_DIR,"adapter"))
model.config.use_cache = False
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
model.config._attn_implementation = "eager"

print("Adapter loaded. Embeddings rows:", model.get_input_embeddings().num_embeddings)


Resized vocab: 32000 -> 32010
Adapter loaded. Embeddings rows: 32010


In [16]:
# cont training
from pathlib import Path
import torch, gc
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForCausalLM,
    DataCollatorForLanguageModeling, TrainingArguments, Trainer
)
from peft import PeftModel

# config
OUT_DIR     = "/content/wakefyer_out_nobnb"
BASE_MODEL  = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TRAIN_TEXT  = "/content/FinnegansWake.txt"
max_len, stride       = 256, 128
batch_size, grad_accum= 1, 32
lr, log_steps         = 1e-4, 10
epochs_more           = 2
eval_ratio, seed      = 0.02, 42

# tokenizer
tok = AutoTokenizer.from_pretrained(Path(OUT_DIR, "tokenizer"), use_fast=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token
tok.padding_side = "right"

# base model -> resize to tokenizer vocab -> load adapter
config = AutoConfig.from_pretrained(BASE_MODEL)
config.pad_token_id = tok.eos_token_id

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    config=config,
    device_map="auto",
    dtype=torch.float16,   # use 'dtype'
)

old_vocab = base.get_input_embeddings().num_embeddings
new_vocab = len(tok)
if old_vocab != new_vocab:
    # explicit mean-resizing if available; otherwise plain resize
    try:
        base.resize_token_embeddings(new_vocab, mean_resizing=True)
    except TypeError:
        base.resize_token_embeddings(new_vocab)
    base.config.vocab_size = new_vocab
print(f"Resized vocab: {old_vocab} → {new_vocab}")

# Lora stands on my shoulders
model = PeftModel.from_pretrained(base, Path(OUT_DIR, "adapter"))
model.config.use_cache = False
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
model.config._attn_implementation = "eager"

print("Adapter loaded. Embeddings rows:", model.get_input_embeddings().num_embeddings)

# dataset rebuild
raw_txt = Path(TRAIN_TEXT).read_text(encoding="utf-8", errors="ignore")
ids = tok(raw_txt, add_special_tokens=False).input_ids

def sliding_windows(txt_ids, block_size=max_len, step=stride):
    items = []
    for s in range(0, max(1, len(txt_ids)-block_size), step):
        ch = txt_ids[s:s+block_size]
        if ch:
            items.append({"input_ids": ch, "attention_mask": [1]*len(ch)})
    if not items:
        ch = txt_ids[:block_size]
        items = [{"input_ids": ch, "attention_mask": [1]*len(ch)}]
    return Dataset.from_list(items)

ds = sliding_windows(ids, max_len, stride)
test_n = max(1, int(len(ds) * eval_ratio)) if len(ds) > 1 else 1
splits = ds.train_test_split(test_size=test_n, seed=seed) if len(ds) > 1 else {"train": ds, "test": ds}
train_ds, eval_ds = splits["train"], splits["test"]
collator = DataCollatorForLanguageModeling(tok, mlm=False)
print("Windows:", len(ds), "| Train:", len(train_ds), "Eval:", len(eval_ds))

# force the correct TrainingArguments
from transformers.training_args import TrainingArguments as HFTrainingArguments
TrainingArguments = HFTrainingArguments  # alias

args = TrainingArguments(
    output_dir=OUT_DIR,

    num_train_epochs=epochs_more,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,

    learning_rate=lr,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,

    logging_strategy="steps",   # some versions require this explicitly
    logging_steps=log_steps,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=5,

    fp16=True,                  # T4
    save_safetensors=True,
    report_to=[],
)

# dempotent shims
import accelerate
from accelerate import Accelerator
if not getattr(Accelerator, "_wake_patch_unwrap", False):
    Accelerator._wake_orig_unwrap = Accelerator.unwrap_model
    def _wake_unwrap(self, model, *args, **kwargs):
        kwargs.pop("keep_torch_compile", None)
        return Accelerator._wake_orig_unwrap(self, model, *args, **kwargs)
    Accelerator.unwrap_model = _wake_unwrap
    Accelerator._wake_patch_unwrap = True
    print("Patched accelerate.Accelerator.unwrap_model (idempotent).")

import transformers.trainer_pt_utils as pt_utils
if not getattr(pt_utils, "_wake_disable_scaler", False):
    pt_utils.get_grad_scaler = lambda *a, **k: None  # prevent fp16 unscale crash
    pt_utils._wake_disable_scaler = True
    print("Disabled Trainer GradScaler hook (idempotent).")

# trainer + go
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collator,
    tokenizer=tok,
)

trainer.train()


Resized vocab: 32000 → 32010
Adapter loaded. Embeddings rows: 32010


Token indices sequence length is longer than the specified maximum sequence length for this model (430235 > 2048). Running this sequence through the model will result in indexing errors
/tmp/ipython-input-202067129.py:126: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Windows: 3360 | Train: 3293 Eval: 67


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

At the representational core sits TinyLlama-1.1B-Chat, deployed under mixed-precision arithmetic (bf16/fp16) to optimize throughput on limited-memory T4 hardware. The model’s internal cache is disabled (use_cache=False), and gradient checkpointing is invoked to trade recomputation for reduced memory allocation. The resultant architecture acts as a self-attenuating transformer: it “forgets” in order to remember within bounds, mirroring the cyclical amnesia of the Wakean text.

The system applies Low-Rank Adaptation (LoRA) across attention and MLP projection submodules (q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj) thereby constraining fine-tuning to a low-dimensional manifold within parameter space. This selective plasticity produces what might be termed controlled stylistic drift: the model learns Joyce’s texture rather than his lexicon. The adapters’ hyperparameters (r=16, α=32, dropout=0.05) instantiate a medium-temperature update regime balancing expressivity and stability. Conceptually, LoRA functions here as a poetic prosthesis, a syntactic exoskeleton grafted onto the pretrained linguistic body.

Due to the GPU’s constrained VRAM capacity, each forward–backward pass processes a micro-batch of size 1, while gradients are accumulated over 32 iterations before a single optimization step. This simulates a virtual batch of 32 without exceeding hardware limits. Technically, this implements delayed gradient aggregation; rhetorically, it allegorizes the model’s incremental cognition: many small thoughts precipitating one grand revelation. The accumulation mechanism thus becomes both a computational strategy and a metaphor for compositional patience.

Training operates within an extended epochic horizon (epochs=1 but looping across numerous windows) using a cosine-annealed learning rate (lr=1e-4, warmup_ratio=0.03). Frequent checkpointing establishes a temporal ecology of model states, allowing resumption under volatile runtime constraints. The “save-steps” routine externalizes the process’s fragmentary nature (mirroring textual serialization in Work in Progress) by preserving multiple partial selves of the model as artifacts of its ongoing metamorphosis.

Inference employs a stochastic sampling configuration (temperature = 0.9, top_p = 0.95) optimized for stylistic exuberance rather than lexical precision. The generation prompt functions as both benchmark and incantation, a recursive self-reference to the corpus origin. Output decoding omits special tokens, yielding the model’s pure linguistic effervescence.

Technically a fine-tuned transformer; conceptually, a computational poetics experiment exploring how ml architectures can approximate literary un-readability. The Wakefyer thus redefines “loss” not as degradation but as aesthetic principle: every rounding error, every truncated sequence, every low-rank projection is a microcosm of Joyce’s deliberate semantic noise. The pipeline enacts a theory of productive distortion, situating training itself as a form of creative misprision.

to be continued